# 🧪 Detection Experiments: Testing AI Limits & Capabilities

**Time to be a scientist!** 🔬 Let's systematically test what affects AI detection and discover the boundaries of what's possible.

## 🎯 What You'll Discover
- How distance affects detection accuracy
- The impact of lighting conditions
- Object orientation and angle effects
- Size limitations and optimal conditions
- Environmental factors that help or hurt AI

## 🔬 Scientific Method Applied to AI
We'll use **controlled experiments** to understand AI behavior:
1. **Hypothesis**: Make a prediction
2. **Test**: Try it with the camera
3. **Observe**: Record what happens
4. **Analyze**: Understand why
5. **Apply**: Use knowledge for better AI

---

## 🔧 Experiment Setup

Let's prepare our experimental environment:

In [ ]:
import subprocess
import cv2
import numpy as np
from IPython.display import display, clear_output, Image, HTML
import ipywidgets as widgets
import threading
import time
import matplotlib.pyplot as plt
from datetime import datetime

# Experiment tracking variables
experiment_results = []
current_experiment = None

print("🧪 AI DETECTION EXPERIMENT LAB")
print("=" * 35)
print("\n🔬 Welcome, AI Researcher!")
print("\n📋 Available Experiments:")
print("   1. 📏 Distance Testing")
print("   2. 💡 Lighting Conditions")
print("   3. 🔄 Object Orientation")
print("   4. 📐 Size & Scale Effects")
print("   5. 🌫️ Environmental Challenges")
print("   6. ⚡ Performance Optimization")

# Setup experiment controls
experiment_selector = widgets.Dropdown(
    options=[
        ('📏 Distance Testing', 'distance'),
        ('💡 Lighting Conditions', 'lighting'),
        ('🔄 Object Orientation', 'orientation'),
        ('📐 Size & Scale Effects', 'scale'),
        ('🌫️ Environmental Challenges', 'environment'),
        ('⚡ Performance Optimization', 'performance')
    ],
    value='distance',
    description='Experiment:',
    style={'description_width': 'initial'}
)

start_experiment_btn = widgets.Button(
    description='🚀 Start Experiment',
    button_style='success',
    layout=widgets.Layout(width='150px', height='40px')
)

stop_experiment_btn = widgets.Button(
    description='⏹️ Stop',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

experiment_output = widgets.Output()
video_output = widgets.Output()

display(widgets.VBox([
    experiment_selector,
    widgets.HBox([start_experiment_btn, stop_experiment_btn]),
    experiment_output,
    video_output
]))

print("\n✅ Experiment lab ready!")
print("\n🎯 Choose an experiment above and click 'Start Experiment'")

In [ ]:
# Experiment definitions and procedures
EXPERIMENTS = {
    'distance': {
        'name': '📏 Distance Testing Experiment',
        'hypothesis': 'Objects closer to camera will have higher confidence scores',
        'procedure': [
            '1. Place object very close to camera (< 1 foot)',
            '2. Slowly move object away from camera',
            '3. Note confidence changes at different distances',
            '4. Find maximum reliable detection distance'
        ],
        'what_to_observe': 'Confidence scores decreasing with distance',
        'tips': 'Use a distinctive object like a coffee cup or book'
    },
    'lighting': {
        'name': '💡 Lighting Conditions Experiment',
        'hypothesis': 'Better lighting improves detection accuracy and confidence',
        'procedure': [
            '1. Test in bright natural light',
            '2. Test in dim indoor lighting',
            '3. Test with artificial light sources',
            '4. Test in shadows or backlighting'
        ],
        'what_to_observe': 'How confidence scores change with light levels',
        'tips': 'Try positioning lamp or moving near window'
    },
    'orientation': {
        'name': '🔄 Object Orientation Experiment',
        'hypothesis': 'Objects shown from typical viewing angles detect better',
        'procedure': [
            '1. Show object from normal viewing angle',
            '2. Rotate object 90 degrees',
            '3. Show object upside down',
            '4. Show object from unusual angles'
        ],
        'what_to_observe': 'Which orientations give highest confidence',
        'tips': 'Books, phones, and cups work great for this test'
    },
    'scale': {
        'name': '📐 Size & Scale Effects Experiment',
        'hypothesis': 'Medium-sized objects in frame detect better than very large or tiny ones',
        'procedure': [
            '1. Fill entire frame with object (very close)',
            '2. Show object taking ~50% of frame',
            '3. Show object taking ~25% of frame',
            '4. Show object very small in frame'
        ],
        'what_to_observe': 'Optimal size for detection confidence',
        'tips': 'Use zoom or distance to control apparent size'
    },
    'environment': {
        'name': '🌫️ Environmental Challenges Experiment',
        'hypothesis': 'Cluttered backgrounds and occlusion reduce detection performance',
        'procedure': [
            '1. Test with clean, simple background',
            '2. Test with cluttered background',
            '3. Test with similar objects nearby',
            '4. Test with partial occlusion (object partially hidden)'
        ],
        'what_to_observe': 'How background complexity affects detection',
        'tips': 'Try placing object on desk vs. in cluttered area'
    },
    'performance': {
        'name': '⚡ Performance Optimization Experiment',
        'hypothesis': 'Lower resolution and framerate can maintain accuracy while improving stability',
        'procedure': [
            '1. Test at full resolution (1920x1080)',
            '2. Test at medium resolution (1280x720)',
            '3. Test at lower framerate (15fps vs 30fps)',
            '4. Monitor CPU temperature during tests'
        ],
        'what_to_observe': 'Performance vs accuracy trade-offs',
        'tips': 'Check system temperature: vcgencmd measure_temp'
    }
}

def start_experiment(experiment_type):
    """Start the selected experiment"""
    global current_experiment
    current_experiment = experiment_type
    
    exp_info = EXPERIMENTS[experiment_type]
    
    with experiment_output:
        clear_output(wait=True)
        print(f"🧪 {exp_info['name']}")
        print("=" * 50)
        print(f"\n🎯 HYPOTHESIS: {exp_info['hypothesis']}")
        print(f"\n📋 PROCEDURE:")
        for step in exp_info['procedure']:
            print(f"   {step}")
        print(f"\n👀 WHAT TO OBSERVE: {exp_info['what_to_observe']}")
        print(f"\n💡 TIPS: {exp_info['tips']}")
        print(f"\n🚀 Experiment starting in 3 seconds...")
        print(f"\n📊 Watch for confidence score changes in the video feed below!")
    
    # Start video feed after short delay
    time.sleep(3)
    start_detection_feed(experiment_type)

def start_detection_feed(experiment_type):
    """Start AI detection video feed for experiments"""
    
    # Choose resolution based on experiment
    if experiment_type == 'performance':
        width, height, fps = "1280", "720", "20"  # Lower for performance testing
    else:
        width, height, fps = "1920", "1080", "30"  # Full quality for other tests
    
    # Clean up existing processes
    for p in ("rpicam-vid", "rpicam-still"):
        subprocess.run(["pkill", "-9", p], stderr=subprocess.DEVNULL)
    
    def run_detection():
        cmd = [
            "rpicam-vid",
            "--inline",
            "-t", "0",
            "--width", width,
            "--height", height,
            "--framerate", fps,
            "--codec", "mjpeg",
            "--post-process-file", "/usr/share/rpi-camera-assets/imx500_mobilenet_ssd.json",
            "-o", "-"
        ]
        
        try:
            proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            data = b""
            frame_count = 0
            start_time = time.time()
            
            display_handle = None
            
            while current_experiment == experiment_type:
                chunk = proc.stdout.read(4096)
                if not chunk:
                    break
                    
                data += chunk
                
                start = data.find(b'\xff\xd8')
                end = data.find(b'\xff\xd9')
                
                if start != -1 and end != -1:
                    jpg_data = data[start:end+2]
                    data = data[end+2:]
                    
                    try:
                        frame = cv2.imdecode(np.frombuffer(jpg_data, dtype=np.uint8), cv2.IMREAD_COLOR)
                        if frame is not None:
                            _, display_jpg = cv2.imencode('.jpg', frame, [cv2.IMWRITE_JPEG_QUALITY, 85])
                            
                            with video_output:
                                if display_handle is None:
                                    display_handle = display(Image(data=display_jpg.tobytes()), display_id=True)
                                else:
                                    display_handle.update(Image(data=display_jpg.tobytes()))
                            
                            frame_count += 1
                            
                            # Update experiment status every 60 frames
                            if frame_count % 60 == 0:
                                elapsed = time.time() - start_time
                                fps_actual = frame_count / elapsed
                                
                                with experiment_output:
                                    # Keep existing content, just update status
                                    print(f"\n⏱️ Experiment Status: Running for {elapsed:.0f}s at {fps_actual:.1f} FPS")
                                    if experiment_type == 'performance':
                                        # Show CPU temp for performance experiments
                                        try:
                                            temp_result = subprocess.run(['vcgencmd', 'measure_temp'], 
                                                                       capture_output=True, text=True)
                                            if temp_result.returncode == 0:
                                                temp = temp_result.stdout.strip().split('=')[1]
                                                print(f"🌡️ CPU Temperature: {temp}")
                                        except:
                                            pass
                    except:
                        continue
                
                time.sleep(0.001)
                
        except Exception as e:
            with experiment_output:
                print(f"\n❌ Experiment error: {str(e)}")
        finally:
            try:
                proc.terminate()
                proc.wait(timeout=2)
            except:
                proc.kill()
    
    # Run detection in separate thread
    detection_thread = threading.Thread(target=run_detection, daemon=True)
    detection_thread.start()

def stop_experiment():
    """Stop current experiment"""
    global current_experiment
    current_experiment = None
    
    # Kill camera processes
    for p in ("rpicam-vid", "rpicam-still"):
        subprocess.run(["pkill", "-9", p], stderr=subprocess.DEVNULL)
    
    with experiment_output:
        print("\n⏹️ Experiment stopped")
        print("\n📝 Don't forget to record your observations!")
        print("\n🔬 Ready for next experiment or analysis")
    
    with video_output:
        clear_output(wait=True)

# Connect button handlers
def on_start_click(button):
    start_experiment(experiment_selector.value)

def on_stop_click(button):
    stop_experiment()

start_experiment_btn.on_click(on_start_click)
stop_experiment_btn.on_click(on_stop_click)

print("\n✅ Experiment controls ready!")
print("\n🎯 Select an experiment type and click 'Start Experiment' to begin")

## 📊 Experiment Results Recording & Analysis

Use this section to record your observations and analyze patterns:

In [ ]:
# Experiment results recording system
import json
from datetime import datetime

print("📊 EXPERIMENT RESULTS LAB")
print("=" * 30)

# Sample results from common experiments
sample_results = {
    "📏 Distance Testing": {
        "observations": {
            "Very Close (< 1 foot)": "95-99% confidence, object may be cropped",
            "Optimal Range (1-4 feet)": "85-95% confidence, best detection",
            "Medium Distance (4-8 feet)": "70-85% confidence, still reliable",
            "Far Distance (8+ feet)": "40-70% confidence, depends on object size",
            "Very Far (15+ feet)": "<40% confidence, unreliable"
        },
        "key_findings": [
            "Optimal detection range: 1-6 feet",
            "Large objects detectable further than small ones",
            "Confidence drops exponentially with distance"
        ]
    },
    "💡 Lighting Conditions": {
        "observations": {
            "Bright Natural Light": "90-95% confidence, excellent accuracy",
            "Good Indoor Lighting": "80-90% confidence, very good",
            "Dim Indoor Light": "60-80% confidence, acceptable",
            "Shadows/Backlighting": "40-60% confidence, challenging",
            "Very Dark": "<40% confidence, unreliable"
        },
        "key_findings": [
            "Natural light gives best results",
            "Shadows significantly impact confidence",
            "Even overhead light improves detection dramatically"
        ]
    },
    "🔄 Object Orientation": {
        "observations": {
            "Normal Viewing Angle": "85-95% confidence",
            "45° Rotation": "75-85% confidence",
            "90° Rotation": "60-75% confidence",
            "Upside Down": "50-70% confidence",
            "Extreme Angles": "30-50% confidence"
        },
        "key_findings": [
            "AI trained on typical viewing angles",
            "Some objects more orientation-sensitive than others",
            "Symmetrical objects less affected by rotation"
        ]
    }
}

# Display sample results
for experiment, results in sample_results.items():
    print(f"\n🧪 {experiment}")
    print("-" * 40)
    
    print("\n📋 Typical Observations:")
    for condition, result in results["observations"].items():
        print(f"   • {condition}: {result}")
    
    print("\n🎯 Key Findings:")
    for finding in results["key_findings"]:
        print(f"   ✓ {finding}")

print("\n\n📝 YOUR EXPERIMENT NOTES:")
print("=" * 30)
print("Use the space below to record your own observations:")

# Create interactive note-taking area
notes_area = widgets.Textarea(
    value="Experiment: \nDate: {}\n\nObservations:\n- \n- \n- \n\nConclusions:\n- \n- \n\nNext steps: \n".format(datetime.now().strftime("%Y-%m-%d %H:%M")),
    placeholder='Record your experiment observations here...',
    description='Lab Notes:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

save_notes_btn = widgets.Button(
    description='💾 Save Notes',
    button_style='info',
    layout=widgets.Layout(width='120px')
)

def save_notes(button):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"/tmp/ai_experiment_notes_{timestamp}.txt"
    
    try:
        with open(filename, 'w') as f:
            f.write(notes_area.value)
        print(f"✅ Notes saved to {filename}")
    except Exception as e:
        print(f"❌ Error saving notes: {e}")

save_notes_btn.on_click(save_notes)

display(widgets.VBox([notes_area, save_notes_btn]))

## 🎯 Quick Experiment Challenges

Try these focused challenges to test specific aspects of AI detection:

In [ ]:
print("🏆 AI DETECTION CHALLENGES")
print("=" * 30)
print("\n🎮 Complete these challenges to master AI detection:")

challenges = [
    {
        "title": "🎯 The Confidence Hunter",
        "task": "Find an object that consistently gives >95% confidence",
        "difficulty": "Easy",
        "tips": "Try well-lit, distinctive objects like phones or cups",
        "success_criteria": "Achieve >95% confidence for 10+ seconds"
    },
    {
        "title": "📏 The Distance Master",
        "task": "Find the maximum distance for reliable detection",
        "difficulty": "Medium",
        "tips": "Start close and slowly move away while watching confidence",
        "success_criteria": "Determine max distance for >70% confidence"
    },
    {
        "title": "🌙 The Shadow Detective",
        "task": "Get good detection in challenging lighting",
        "difficulty": "Medium",
        "tips": "Try different light sources and angles",
        "success_criteria": "Achieve >80% confidence in dim conditions"
    },
    {
        "title": "🔄 The Orientation Optimizer",
        "task": "Find the best angle for tricky objects",
        "difficulty": "Medium",
        "tips": "Test books, remotes, or phones at different angles",
        "success_criteria": "Compare confidence at 5+ different angles"
    },
    {
        "title": "🏃 The Speed Demon",
        "task": "Test detection on moving objects",
        "difficulty": "Hard",
        "tips": "Move objects at different speeds across the frame",
        "success_criteria": "Track object successfully while in motion"
    },
    {
        "title": "🎭 The Impostor Hunter",
        "task": "Find objects that fool the AI",
        "difficulty": "Hard",
        "tips": "Try objects that look similar to COCO dataset items",
        "success_criteria": "Find 3+ objects that get misidentified"
    }
]

for i, challenge in enumerate(challenges, 1):
    difficulty_emoji = {
        "Easy": "🟢",
        "Medium": "🟡", 
        "Hard": "🔴"
    }[challenge["difficulty"]]
    
    print(f"\n{i}. {challenge['title']} {difficulty_emoji}")
    print(f"   📋 Task: {challenge['task']}")
    print(f"   💡 Tips: {challenge['tips']}")
    print(f"   ✅ Success: {challenge['success_criteria']}")

print("\n\n🏅 CHALLENGE TRACKING:")
print("-" * 20)
print("Mark your completed challenges:")
print("□ Confidence Hunter")
print("□ Distance Master")
print("□ Shadow Detective")
print("□ Orientation Optimizer")
print("□ Speed Demon")
print("□ Impostor Hunter")

print("\n🎉 Complete all 6 challenges to become an AI Detection Expert!")
print("\n💪 Pro Tip: Document your results in the notes section above!")

## 📈 Performance Analysis & Optimization Tips

Learn how to optimize AI performance for different scenarios:

In [ ]:
print("⚡ AI PERFORMANCE OPTIMIZATION GUIDE")
print("=" * 40)

print("\n🎯 OPTIMIZATION STRATEGIES:")
print("-" * 30)

optimization_tips = {
    "🖥️ For Pi Zero 2W": {
        "resolution": "Use 1280x720 for complex applications",
        "framerate": "20fps is sufficient for most uses",
        "cooling": "Add passive heatsink for sustained operation",
        "os": "Use Pi OS Lite (64-bit) - no desktop"
    },
    "📊 For Accuracy": {
        "lighting": "Ensure good, even lighting",
        "distance": "Keep objects 1-6 feet from camera",
        "background": "Use simple, uncluttered backgrounds",
        "stability": "Mount camera to reduce motion blur"
    },
    "⚡ For Speed": {
        "resolution": "Lower resolution = faster processing",
        "roi": "Focus on specific region of interest",
        "threshold": "Higher confidence threshold = fewer detections to process",
        "objects": "Filter to only objects you care about"
    },
    "🔋 For Battery Life": {
        "framerate": "Reduce to 10-15fps for battery applications",
        "resolution": "640x480 can be sufficient for basic detection",
        "sleep": "Use detection intervals instead of continuous",
        "gpio": "Power down unnecessary peripherals"
    }
}

for category, tips in optimization_tips.items():
    print(f"\n{category}:")
    for aspect, tip in tips.items():
        print(f"   • {aspect.title()}: {tip}")

print("\n\n🌡️ THERMAL MANAGEMENT:")
print("-" * 25)
print("Monitor CPU temperature to prevent throttling:")
print("")
print("# Check current temperature")
print("vcgencmd measure_temp")
print("")
print("# Temperature guidelines:")
print("• <60°C: Excellent")
print("• 60-70°C: Good")
print("• 70-80°C: Monitor closely")
print("• >80°C: Add cooling or reduce load")

print("\n\n📊 PERFORMANCE METRICS TO TRACK:")
print("-" * 35)
metrics = [
    "🖼️ Actual FPS achieved",
    "🌡️ CPU temperature",
    "📊 Average confidence scores",
    "⏱️ Detection latency",
    "🎯 Detection accuracy rate",
    "💾 Memory usage"
]

for metric in metrics:
    print(f"   {metric}")

print("\n\n🔧 QUICK PERFORMANCE CHECK:")
print("-" * 30)
try:
    # Check CPU temperature
    temp_result = subprocess.run(['vcgencmd', 'measure_temp'], capture_output=True, text=True)
    if temp_result.returncode == 0:
        temp = temp_result.stdout.strip()
        print(f"🌡️ Current CPU Temperature: {temp}")
        
        temp_val = float(temp.split('=')[1].replace("'C", ""))
        if temp_val < 60:
            print("   ✅ Excellent - No thermal concerns")
        elif temp_val < 70:
            print("   🟡 Good - Normal operating temperature")
        elif temp_val < 80:
            print("   🟠 Warm - Monitor during extended use")
        else:
            print("   🔴 Hot - Consider adding cooling or reducing load")
    else:
        print("❌ Unable to read CPU temperature")
except Exception as e:
    print(f"❌ Temperature check failed: {e}")

# Check available memory
try:
    mem_result = subprocess.run(['free', '-h'], capture_output=True, text=True)
    if mem_result.returncode == 0:
        mem_lines = mem_result.stdout.strip().split('\n')
        if len(mem_lines) >= 2:
            mem_info = mem_lines[1].split()
            print(f"💾 Memory: {mem_info[2]} used / {mem_info[1]} total")
except Exception as e:
    print(f"❌ Memory check failed: {e}")

print("\n🎯 Remember: Optimization is about finding the right balance for YOUR specific use case!")

## 🎉 Experiment Conclusions

Congratulations on completing your AI detection experiments! Let's summarize what you've learned:

In [ ]:
print("🎓 EXPERIMENT CONCLUSIONS & KEY LEARNINGS")
print("=" * 45)

key_discoveries = {
    "🔍 Detection Fundamentals": [
        "AI confidence varies dramatically with conditions",
        "Distance, lighting, and orientation all matter",
        "Optimal detection zone: 1-6 feet with good lighting",
        "Background complexity affects detection reliability"
    ],
    "⚡ Performance Insights": [
        "IMX500 does AI processing, Pi CPU stays cool",
        "Resolution and framerate can be tuned for application",
        "Pi Zero 2W can handle 30fps at 1080p for simple scenes",
        "Thermal management important for sustained operation"
    ],
    "🎯 Practical Applications": [
        "Choose confidence threshold based on use case",
        "Environmental factors must be considered in design",
        "Testing and validation essential for reliability",
        "Optimization requires balancing multiple factors"
    ],
    "🧠 AI Understanding": [
        "AI has predictable strengths and limitations",
        "Training data affects what AI can detect well",
        "Confidence scores provide valuable reliability information",
        "Real-world performance differs from lab conditions"
    ]
}

for category, learnings in key_discoveries.items():
    print(f"\n{category}:")
    for learning in learnings:
        print(f"   ✓ {learning}")

print("\n\n🚀 NEXT STEPS IN YOUR AI JOURNEY:")
print("-" * 35)
next_steps = [
    "📚 Study Object Types Guide to understand all 80+ detectable objects",
    "🔊 Move to Level 3: Interactive AI with GPIO and buzzer alerts",
    "🚗 Explore Level 4: Integration with VESC motor data",
    "🏗️ Design Level 5: Your own real-world AI application",
    "🔬 Continue experimenting with different objects and scenarios",
    "📊 Practice choosing appropriate confidence thresholds"
]

for step in next_steps:
    print(f"   {step}")

print("\n\n🏆 EXPERIMENT MASTER ACHIEVEMENT UNLOCKED!")
print("=" * 45)
print("You now have hands-on experience with:")
print("   🔬 Scientific method applied to AI")
print("   📊 Systematic performance analysis")
print("   🎯 Optimization for real-world conditions")
print("   🧠 Deep understanding of AI behavior")

print("\n🎓 You're ready to build reliable AI applications!")
print("\n💡 Pro Tip: Keep experimenting! Every new scenario teaches you something about AI behavior.")

## 🚀 What's Next?

Amazing work completing these detection experiments! You now have a **scientific understanding** of AI behavior.

### 📚 Continue Your AI Journey:

**Next Notebook**: `Object_Types_Guide.ipynb`
- Complete reference to all 80+ detectable objects
- Tips for detecting specific object categories
- Understanding the COCO dataset

**Ready for Level 3?**: `../03_Interactive_AI/AI_with_Buzzer_Alerts.ipynb`
- Make AI respond to the world with sound and GPIO!
- Build your first interactive AI system
- Learn hardware integration patterns

### 🎯 Key Achievements Unlocked

✅ **Scientific AI Analysis**: You can systematically test AI performance

✅ **Performance Optimization**: You understand the trade-offs and tuning parameters

✅ **Real-World Readiness**: You know how environmental factors affect AI

✅ **Reliability Assessment**: You can evaluate when AI results are trustworthy

---

### 🧠 You're Now an AI Experimentalist!

You have the skills to **test**, **validate**, and **optimize** AI systems for real-world deployment. This experimental mindset will serve you well as you build increasingly sophisticated AI applications! 🤖🔬✨